## 01. Introduction Homework

In [11]:
# import libraries
import numpy as np
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

#### Q1. Downloading the data

In [12]:
df_jan_2023 = pd.read_parquet('../../data/yellow_tripdata_2023-01.parquet')
df_feb_2023 = pd.read_parquet('../../data/yellow_tripdata_2023-02.parquet')

print(f"jan 2023: {len(df_jan_2023.columns)}, feb 2023: {len(df_feb_2023.columns)}.")

jan 2023: 19, feb 2023: 19.


#### Q2. Computing duration

In [13]:
df_jan_2023['duration'] = df_jan_2023.tpep_dropoff_datetime - df_jan_2023.tpep_pickup_datetime
df_jan_2023.duration = df_jan_2023.duration.apply(lambda td: td.total_seconds() / 60)

df_feb_2023['duration'] = df_feb_2023.tpep_dropoff_datetime - df_feb_2023.tpep_pickup_datetime
df_feb_2023.duration = df_feb_2023.duration.apply(lambda td: td.total_seconds() / 60)

std_dev = df_jan_2023['duration'].std()

print(np.round(std_dev, 2))

42.59


#### Q3. Dropping outliers

In [14]:
# filter the DataFrame to remove outliers
df_jan_filtered = df_jan_2023[(df_jan_2023['duration'] >= 1) & (df_jan_2023['duration'] <= 60)]
df_feb_filtered = df_feb_2023[(df_feb_2023['duration'] >= 1) & (df_feb_2023['duration'] <= 60)]

# calculate the fraction of the records left
fraction_left = len(df_jan_filtered) / len(df_jan_2023)

print(fraction_left)

0.9812202822125979


#### Q4. One-hot encoding

In [15]:
# convert the dataframe to a list of dictionaries
train_data_dict = df_jan_filtered[['PULocationID', 'DOLocationID']].astype(str).to_dict('records')

# initialize the DictVectorizer
vec = DictVectorizer()

# fit and transform the data
feature_matrix = vec.fit_transform(train_data_dict)

print(f"the dimensionality of the matrix is: {feature_matrix.shape[1]}")

the dimensionality of the matrix is: 515


#### Q5. One-hot encoding

In [16]:
# initialize the linear regression model
model = LinearRegression()

# fit the model on the feature matrix and the target variable
model.fit(feature_matrix, df_jan_filtered['duration'])

# predict the target variable on the training data
train_preds = model.predict(feature_matrix)

# calculate the RMSE on the training data
rmse_train = np.sqrt(mean_squared_error(df_jan_filtered['duration'], train_preds))

print(f"the RMSE on the training data is: {rmse_train}")

the RMSE on the training data is: 7.649261936284003


#### Q6. Evaluating the model

In [17]:
# convert the validation dataframe to a list of dictionaries
val_data_dict = df_feb_2023[['PULocationID', 'DOLocationID']].astype(str).to_dict('records')

# transform the validation data using the same DictVectorizer
val_feature_matrix = vec.transform(val_data_dict)

# predict the target variable on the validation data
val_preds = model.predict(val_feature_matrix)

# calculate the RMSE on the validation data
rmse_val = np.sqrt(mean_squared_error(df_feb_2023['duration'], val_preds))

print(f"the RMSE on the validation data is: {rmse_val}")

the RMSE on the validation data is: 42.28477602158213
